# Black-Scholes

## Sommaire :

* [**1.Modèle**](#0)

* [**2.Calibration**](#1)

* [**3.Validation**](#2)

In [17]:
# Modules 
import requests  # Connexion à l'API de AlphaVantage
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from dataclasses import dataclass, field
from __future__ import annotations
from dataclasses import dataclass, field
from scipy import stats

In [28]:
# Clé personnelle de l'API AlphaVantage
key_API = 'LYMJQ6KR5QPKJ8W3'
equity = 'IBM'   # choisir n'importe quelle equity

# URL pour obtenir les données hebdomadaires ajustées
url = f'https://www.alphavantage.co/query?function=TIME_SERIES_WEEKLY_ADJUSTED&symbol={equity}&apikey={key_API}'
r = requests.get(url)
data = r.json()

# Accès au sous-dictionnaire contenant les données
time_serie = data['Weekly Adjusted Time Series']

# Extraction des données de clôture 
df = pd.DataFrame({
    "Date": pd.to_datetime(list(time_serie.keys())),
    "Stock Value": [float(entry["4. close"]) for entry in time_serie.values()]
})

<a id='0'></a>
## 1. Modèle :


Soit $(\Omega, \mathcal{F}, \mathbb{P})$ un espace probabilisé, soit $(\mathcal{W}_t)_{t>0}$ un mouvement brownien sous $\mathbb{P}$ et $(\mathcal{S}_t)_{t>0}$ représentant la trajectoire d'un actif.

Sous $\mathbb{P}$, la dynamique de $\mathcal{S}$ s'écrit :

$$
\frac{dS_t}{S_t} = \mu \, dt + \sigma \, dW_t
$$

**Solution de l'EDS :** (Pas de schéma nécessaire)

$$
d(\ln(S_t)) = \frac{1}{S_t} dS_t - \frac{1}{2} \frac{1}{S_t^2} (dS_t)^2
\implies d(\ln(S_t)) = \frac{1}{S_t} \big(S_t (\mu  \, dt + \sigma \, dW_t)\big) - \frac{1}{2} \sigma^2 dt
\implies d(\ln(S_t)) = \left(\mu  - \frac{1}{2} \sigma^2\right) dt + \sigma dW_t
$$

$$
\ln(S_t) = \ln(S_0) + \left(\mu  - \frac{1}{2} \sigma^2\right)t + \sigma W_t
\implies S_t = S_0 \exp\left(\left(\mu  - \frac{1}{2} \sigma^2\right)t + \sigma W_t\right)
$$


$$
\implies S_{t_k} = S_{t_{k-1}} \exp\left(\left(\mu  - \frac{1}{2} \sigma^2\right)(t_k - t_{k-1}) + \sigma \left(W_{t_k} - W_{t_{k-1}}\right)\right)
$$



$$
\implies
\boxed{ \ln \Big(\tfrac{S_{t_k}}{S_{t_{k-1}}}\Big) \,\big|\, \mathcal{F}_s 
\sim \mathcal{N}\!\Big( \big(\mu-\tfrac12\sigma^2\big)(t_k - t_{k-1}),\; \sigma^2(t_k - t_{k-1}) \Big) }
$$

---

<a id='1'></a>
## 2. Calibration :

In [ ]:
@dataclass
class BlackScholes:
    """
    Modèle Black–Scholes (GBM) pour un actif S_t.

    Attributs
    ---------
    S0 : float | None
        Dernier prix observé de l’actif.
    mu : float | None
        Drift sous la mesure historique P (continu, annualisé).
    sigma : float | None
        Volatilité de l’actif (continue, annualisée).
    q : float
        Taux de dividende continu (annualisé).
    dt : float
        Pas de temps en années des données historiques (ex: 1/252).
    meta : dict
        Dictionnaire pour stocker des informations de calibration.
    """
    S0: float | None = None
    mu: float | None = None
    sigma: float | None = None
    q: float = 0.0
    dt: float = 1/252
    meta: dict = field(default_factory=dict)

    # ---------- UTILITAIRES ----------
    @staticmethod
    def _to_log_returns(prices: np.ndarray) -> np.ndarray:
        """
        Calcule les log-returns consécutifs : ln(S_t) - ln(S_{t-1}).

        Paramètres
        ----------
        prices : np.ndarray
            Tableau 1D des prix de l’actif (strictement positifs).

        Retour
        ------
        np.ndarray
            Tableau 1D des log-returns, de longueur len(prices)-1.
        """
        p = np.asarray(prices, float)
        return np.diff(np.log(p))

    @staticmethod
    def infer_dt_from_dates(dates) -> float:
        """
        Infère le pas moyen dt (en années) à partir d'une série de dates.

        Paramètres
        ----------
        dates : array-like
            Liste ou série de dates (convertissable en pandas.DatetimeIndex).

        Retour
        ------
        float
            Pas moyen dt exprimé en années (≈ années_totales / (n_points - 1)).
        """
        import pandas as pd
        s = pd.to_datetime(dates).dropna().sort_values()
        total_years = (s.iloc[-1] - s.iloc[0]) / pd.Timedelta(days=365.25)
        return float(total_years) / (len(s) - 1)

    # ---------- CALIBRATION ----------
    def fit_from_prices(self, prices: np.ndarray, keep_S0: bool = True) -> tuple[float, float]:
        """
        Calibre mu et sigma du modèle par MLE à partir d'une série de prix.

        Paramètres
        ----------
        prices : np.ndarray
            Tableau 1D des prix de l’actif (strictement positifs).
        keep_S0 : bool, défaut True
            - True : conserve self.S0 si déjà défini.
            - False : met self.S0 au dernier prix observé.

        Retour
        ------
        tuple[float, float]
            (mu, sigma) estimés, annualisés.
        """
        lr = self._to_log_returns(prices)
        if self.S0 is None or not keep_S0:
            self.S0 = float(prices[0])

        m = float(np.mean(lr))         # E[Δln S] = (mu - 0.5 sigma^2) dt
        v = float(np.var(lr))          # Var[Δln S] = sigma^2 dt

        sigma_hat = np.sqrt(v / self.dt)
        mu_hat = (m / self.dt) + 0.5 * sigma_hat**2

        self.mu, self.sigma = float(mu_hat), float(sigma_hat)
        self.meta.update({"method": "historical_mle", "n_obs": len(lr), "dt": self.dt})
        return self.mu, self.sigma

    # ---------- SIMULATION ----------
    def simulate_paths(
        self,
        T: float,
        n_steps: int = 252,
        S0: float | None = None,
        n_paths: int = 1000,
        seed: int | None = None,
        rate_curve: np.ndarray | None = None,
        emm: bool = False,
    ) -> np.ndarray:
        """
        Simule des trajectoires de S_t selon un GBM.

        Paramètres
        ----------
        T : float
            Horizon de simulation (en années).
        n_steps : int, défaut 252
            Nombre de pas de temps sur [0, T].
        S0 : float | None, défaut None
            Prix initial de l’actif. Si None, utilise self.S0.
        n_paths : int, défaut 1000
            Nombre de trajectoires simulées.
        seed : int | None, défaut None
            Graine pour le générateur aléatoire (reproductibilité).
        rate_curve : np.ndarray | None
            Courbe zéro-coupon R(0,k) (taux continus) donnée pour k=1..K (années entières).
            Utilisée si emm=True.
        emm : bool, défaut False
            - False : simulation sous P (mesure historique), drift = mu - q.
            - True : simulation sous Q (mesure risque-neutre), drift = r_t - q.

        Retour
        ------
        np.ndarray
            Matrice (n_paths, n_steps+1) contenant les trajectoires simulées.
        """
        rng = np.random.default_rng(seed)
        S0 = self.S0 if S0 is None else float(S0)

        dt = T / n_steps
        vol = self.sigma * np.sqrt(dt)

        paths = np.empty((n_paths, n_steps + 1))
        paths[:, 0] = S0

        if emm:
            # Forwards annuels à partir de R(0,k) (taux continus)
            # f_{0,1} = R(0,1), f_{k-1,k} = k*R(0,k) - (k-1)*R(0,k-1)
            R = np.asarray(rate_curve, float)                     # shape (K,)
            K = len(R)
            forwards = np.empty(K, dtype=float)
            forwards[0] = R[0]
            for k in range(2, K + 1):
                forwards[k - 1] = k * R[k - 1] - (k - 1) * R[k - 2]

            # Pas par an (arrondi simple), répéter chaque forward
            steps_per_year = int(round(n_steps / T))
            n_years_needed = int(np.ceil(T))
            f_yearly = np.concatenate([forwards, np.repeat(forwards[-1], max(0, n_years_needed - K))])[:n_years_needed]
            r_t = np.repeat(f_yearly, steps_per_year)[:n_steps]   # longueur n_steps
            drift_t = (r_t - self.q - 0.5 * self.sigma**2) * dt
        else:
            drift_t = np.full(n_steps, (self.mu - self.q - 0.5 * self.sigma**2) * dt)

        for t in range(1, n_steps + 1):   # à vectoriser pour aller plus vite
            z = rng.standard_normal(n_paths)
            paths[:, t] = paths[:, t - 1] * np.exp(drift_t[t - 1] + vol * z)

        return paths

    # ---------- SÉRIALISATION ----------
    def to_dict(self) -> dict:
        """
        Retourne l'état courant de l’objet sous forme de dictionnaire.

        Retour
        ------
        dict
            {"S0": ..., "mu": ..., "sigma": ..., "q": ..., "dt": ..., "meta": {...}}
        """
        return {"S0": self.S0, "mu": self.mu, "sigma": self.sigma, "q": self.q, "dt": self.dt, "meta": dict(self.meta)}

<a id='2'></a>
## 3. Validation :

In [66]:
# Création + calibration
model = BlackScholes()
model.dt = model.infer_dt_from_dates(df["Date"])
mu, sigma = model.fit_from_prices(df["Stock Value"].values[::-1]) 

# --- Récupérer les paramètres calibrés ---
params = model.to_dict()
S0   = float(params["S0"])
mu   = float(params["mu"])
sigma= float(params["sigma"])
q    = float(params["q"])
dt   = float(params["dt"])

# Paramètres calibrés
params = model.to_dict()
print("Paramètres calibrés :")
for k, v in params.items():
    print(f"  {k} = {v}")

Paramètres calibrés :
  S0 = 95.87
  mu = 0.06864064962644137
  sigma = 0.2562856825729221
  q = 0.0
  dt = 0.019158857846701696
  meta = {'method': 'historical_mle', 'n_obs': 1347, 'dt': 0.019158857846701696}


#### Comparaison des moments

In [68]:
# ---------- Moments empiriques au pas dt ----------
def empirical_moments_dt(log_ret: np.ndarray) -> tuple[float, float]:
    """
    Calcule la moyenne et la variance empiriques de Δln S_t.
    
    Paramètres
    ----------
    log_ret : np.ndarray
        Log-returns au pas dt.
    
    Retour
    ------
    (m_emp, v_emp) : tuple[float, float]
        Moyenne et variance empiriques de Δln S_t.
    """
    m_emp = float(np.mean(log_ret))
    v_emp = float(np.var(log_ret))
    return m_emp, v_emp

# ---------- Moments théoriques au pas dt ----------
def theoretical_moments_dt(mu: float, sigma: float, dt: float) -> tuple[float, float]:
    """
    Moments théoriques de Δln S_t sous GBM : 
    E[Δln S] = (mu - 0.5*sigma^2)*dt, Var[Δln S] = sigma^2*dt.
    
    Paramètres
    ----------
    mu : float
        Drift sous P (continu, annualisé).
    sigma : float
        Volatilité (continue, annualisée).
    dt : float
        Pas de temps en années.
    
    Retour
    ------
    (m_th, v_th) : tuple[float, float]
        Moyenne et variance théoriques de Δln S_t.
    """
    m_th = (mu - 0.5 * sigma**2) * dt
    v_th = (sigma**2) * dt
    return m_th, v_th

In [70]:
prices = df["Stock Value"].values[::-1]  
log_ret = np.diff(np.log(prices))

# moments empiriques
m_emp, v_emp = empirical_moments_dt(log_ret)

# moments théoriques sous GBM
m_th, v_th = theoretical_moments_dt(model.mu, model.sigma, model.dt)

print("empirical_moments_dt :", {"mean": m_emp, "var": v_emp})
print("theoretical_moments_dt :", {"mean": m_th, "var": v_th})

empirical_moments_dt : {'mean': 0.0006858770348951168, 'var': 0.0012583988276062624}
theoretical_moments_dt : {'mean': 0.0006858770348951168, 'var': 0.0012583988276062624}


#### Check résidus

In [73]:
# ---------- Résidus standardisés ----------
def standardized_residuals(log_ret: np.ndarray, m_th: float, v_th: float) -> np.ndarray:
    """
    Résidus standardisés : eps_t = (Δln S_t - m_th) / sqrt(v_th).
    
    Paramètres
    ----------
    log_ret : np.ndarray
        Log-returns observés au pas dt.
    m_th : float
        Moyenne théorique de Δln S_t.
    v_th : float
        Variance théorique de Δln S_t.
    
    Retour
    ------
    np.ndarray
        Résidus standardisés eps_t (attendu ~ N(0,1) i.i.d. sous GBM).
    """
    return (log_ret - m_th) / np.sqrt(v_th)


def residuals_stats(eps: np.ndarray) -> dict:
    """
    Statistiques de base des résidus standardisés.
    
    Paramètres
    ----------
    eps : np.ndarray
        Résidus standardisés.
    
    Retour
    ------
    dict
        {'mean', 'var', 'skew', 'kurt'} (kurtose totale, 3 pour N(0,1)).
    """
    mean = float(np.mean(eps))
    var = float(np.var(eps))
    std = np.sqrt(var)
    skew = float(np.mean(((eps - mean) / std)**3))
    kurt = float(np.mean(((eps - mean) / std)**4))  # kurtose totale
    return {"mean": mean, "var": var, "skew (normal = 0)": skew, "kurt (normal = 3)": kurt}

In [74]:
eps = standardized_residuals(log_ret, m_th, v_th)
stats_eps = residuals_stats(eps)
print("residuals_stats :", stats_eps)

residuals_stats : {'mean': 2.5056258313737757e-17, 'var': 1.0, 'skew (normal = 0)': -0.2252735817980989, 'kurt (normal = 3)': 6.587516730708788}


#### Check simul paths

In [81]:
n_steps = len(log_ret)                 
T = n_steps * model.dt                

# Simuler une trajectoire
paths_1 = model.simulate_paths(T=T, n_steps=n_steps, n_paths=100, seed=42)
sim_prices = paths_1[0]
sim_prices = sim_prices[::-1]  # Penser à inverser
sim_log_ret = np.diff(np.log(sim_prices))

# Moments simulés 
m_sim = float(np.mean(sim_log_ret))
v_sim = float(np.var(sim_log_ret, ddof=1))

# Affichage comparatif
print("Per-step moments comparison")
print(f"- Empirical (history):   mean = {m_emp:.6e}, var = {v_emp:.6e}")
print(f"- Theoretical (GBM):     mean = {m_th:.6e}, var = {v_th:.6e}")
print(f"- Simulated (1 path):    mean = {m_sim:.6e}, var = {v_sim:.6e}")

# Écarts relatifs (par rapport à l'empirique et au théorique)
def rel_err(a, b): 
    return (a - b) / (1e-12 if b == 0 else b)

print("\nRelative errors (vs empirical):")
print(f"  mean: {rel_err(m_sim, m_emp):+.2%}   var: {rel_err(v_sim, v_emp):+.2%}")

print("Relative errors (vs theoretical):")
print(f"  mean: {rel_err(m_sim, m_th):+.2%}    var: {rel_err(v_sim, v_th):+.2%}")

Per-step moments comparison
- Empirical (history):   mean = 6.858770e-04, var = 1.258399e-03
- Theoretical (GBM):     mean = 6.858770e-04, var = 1.258399e-03
- Simulated (1 path):    mean = 2.901899e-04, var = 1.184017e-03

Relative errors (vs empirical):
  mean: -57.69%   var: -5.91%
Relative errors (vs theoretical):
  mean: -57.69%    var: -5.91%


In [86]:
n_steps = len(log_ret)                 
T = n_steps * model.dt                

# Simuler 1000 trajectoires
paths = model.simulate_paths(T=T, n_steps=n_steps, n_paths=10000, seed=42)

# Calcul des log-returns sur toutes les trajectoires
# np.diff marche sur l’axe 1, puis on aplati pour tout agréger
sim_log_ret = np.diff(np.log(paths), axis=1).ravel()

#  Moments simulés (par pas)
m_sim = float(np.mean(sim_log_ret))
v_sim = float(np.var(sim_log_ret, ddof=1))

#  Affichage comparatif
print("Per-step moments comparison (100 paths)")
print(f"- Empirical (history):   mean = {m_emp:.6e}, var = {v_emp:.6e}")
print(f"- Theoretical (GBM):     mean = {m_th:.6e}, var = {v_th:.6e}")
print(f"- Simulated (10000 paths): mean = {m_sim:.6e}, var = {v_sim:.6e}")


Per-step moments comparison (100 paths)
- Empirical (history):   mean = 6.858770e-04, var = 1.258399e-03
- Theoretical (GBM):     mean = 6.858770e-04, var = 1.258399e-03
- Simulated (10000 paths): mean = 6.874280e-04, var = 1.257990e-03
